Start by using either the inspector or by viewing the page source. Can you identify a tag that might be helpful for finding the names of all performers? For now, just worry about the headliner and don't worry about the opener. (Eg. For Vince Gill, featuring Wendy Moten, we only care about Vince Gill.) Make use of this to create a list containing just the names of each inductee.

In [13]:
import requests
import pandas as pd

In [2]:
URL = 'https://www.ryman.com/events/'

response = requests.get(URL)

In [3]:
type(response)

requests.models.Response

In [4]:
response.status_code

200

In [5]:
from bs4 import BeautifulSoup as BS

In [6]:
soup = BS(response.text)

In [7]:
print(soup.prettify())

<!DOCTYPE html>
<!--[if lt IE 7 ]><html lang="en" class="no-js ie6 ie"><![endif]-->
<!--[if IE 7 ]><html lang="en" class="no-js ie7 ie"><![endif]-->
<!--[if (gt IE 9)|!(IE)]><!-->
<html class="no-js" lang="en">
 <!--<![endif]-->
 <head>
  <script>
   if ( /*@cc_on!@*/ false) {
        document.documentElement.className += ' ie10';
    }
    //detect ie11 by CSS Capability Testing
    if ('msTextCombineHorizontal' != undefined) {

        //add class if userAgent isn't Chrome or Safari
        var ua = navigator.userAgent;
        var chromesafari_check = new RegExp("Safari");
        if (chromesafari_check.exec(ua) != null) {
            var isMSIE = false;
        } else {
            var isMSIE = true;
        }

        if (isMSIE) {
            document.documentElement.className += ' ie11';
        }
    }
  </script>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <title>
   Events | Ryman Auditorium
  </title>
  <meta content="Ryman Auditorium

In [15]:
first = soup.select_one("div.thumb a")
act = first['title'].replace('More Info for', '').strip()
print(act)

Ziggy Marley & Burning Spear


In [18]:
performers = []

for a in soup.select('div.thumb a[title]'):
    title = a.get('title', '')
    if title.startswith('More Info for '):
        title = title.replace('More Info for ', '').strip()
    performers.append(title)

performers

['Ziggy Marley & Burning Spear',
 'X (the band) and Los Lobos - CANCELED',
 'Ryman Sidewalk Sessions with Jeff Woods',
 'The Temptations & Four Tops',
 'Ludovico Einaudi',
 'The Prine Family Presents: You Got Gold',
 'Tommy Emmanuel, CGP',
 'A.J. Croce Presents CROCE PLAYS CROCE',
 'An Evening with Patty Griffin + Rickie Lee Jones',
 'MJ Lenderman & The Wind',
 'Noah Cyrus',
 'Ryman Sidewalk Sessions with George Croft']

Next, try and find a tag that could be used to find the date and time for each show. Extract these into a list. Challenge: Convert these into two lists, one containing the date and the other containing the time. (Eg. split Mar 9, 2023 8:00 PM into Mar 9, 2023 and 8:00 PM.)

In [19]:
showdate = [t.get_text(strip=True) for t in soup.findAll('div', class_='date')]
print(type(showdate))
showdate

<class 'list'>


['',
 '',
 'Oct6, 20257:30 PM',
 'Oct7, 20258:00 PM',
 'Oct8, 20255:30 PM',
 'Oct8, 20257:30 PM',
 'Oct9, 20257:30 PM',
 'Oct10, 20257:30 PM',
 'Oct11, 20258:00 PM',
 'Oct12, 20257:30 PM',
 'Oct13, 20258:00 PM',
 'Oct14, 20257:30 PM',
 'Oct15, 20257:30 PM',
 'Oct16, 20256:00 PM']

In [20]:
import re

rows = [s.strip() for s in showdate if s.strip()]

rows = [re.sub(r'^(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(\d{1,2}),', r'\1 \2,', s, flags=re.I) for s in rows]
rows = [re.sub(r'(\d{4})(\d{1,2}:\d{2}\s*[AP]M)', r'\1 \2', s, flags=re.I) for s in rows]

dates, times = [], []
time_re = re.compile(r'(\d{1,2}:\d{2}\s*[AP]M)', re.I)

for s in rows:
    m = time_re.search(s)
    if not m:
        continue
    dates.append(s[:m.start()].strip().rstrip(','))
    times.append(m.group(1).upper())


In [21]:
dates

['Oct 6, 2025',
 'Oct 7, 2025',
 'Oct 8, 2025',
 'Oct 8, 2025',
 'Oct 9, 2025',
 'Oct 10, 2025',
 'Oct 11, 2025',
 'Oct 12, 2025',
 'Oct 13, 2025',
 'Oct 14, 2025',
 'Oct 15, 2025',
 'Oct 16, 2025']

In [22]:
times

['7:30 PM',
 '8:00 PM',
 '5:30 PM',
 '7:30 PM',
 '7:30 PM',
 '7:30 PM',
 '8:00 PM',
 '7:30 PM',
 '8:00 PM',
 '7:30 PM',
 '7:30 PM',
 '6:00 PM']

Take the lists you created on parts 1 and 2 and convert them into a pandas DataFrame.

In [23]:
schedule = pd.DataFrame({
    'Performer': performers,
    'Date': dates,
    'Time': times
})
schedule.head()

,Performer,Date,Time
0,Ziggy Marley & Burning Spear,"Oct 6, 2025",7:30 PM
1,X (the band) and Los Lobos - CANCELED,"Oct 7, 2025",8:00 PM
2,Ryman Sidewalk Sessions with Jeff Woods,"Oct 8, 2025",5:30 PM
3,The Temptations & Four Tops,"Oct 8, 2025",7:30 PM
4,Ludovico Einaudi,"Oct 9, 2025",7:30 PM
